In [1]:
import dimcli
from dimcli.shortcuts import dslquery
import pandas
import time

In [2]:
dimcli.login()

DimCli v0.5.6.1 - Succesfully connected to <https://app.dimensions.ai> (method: dsl.ini file)


In [45]:
df = pandas.read_csv("ISSNsCheckDimensions.csv")

In [46]:
df.head(5)

,Journal Name,ISSN,e-ISSN
0,3 Biotech,2190-572X,2190-5738
1,AAAS MEETING AND SCIENCE INNOVATION EXPOSITION,NaN,NaN
2,AAPS Journal,1550-7416,NaN
3,Abdominal Radiology,2366-004X,2366-0058
4,Abhandlung Naturhistorische Gesellschaft Nuern...,0077-6149,NaN


### Approach

For each row, take the journal name and query Dimensions with it. If there are publications results, add a new cell to the row with the tot pubs and ISSNs. 

An extra step would be to check the ISSNS programmatically, but this can be done manually for now.. 

In [47]:
# create new columns
df['DimensionsPubs'] = None
df['DimensionsIssns'] = None

In [48]:
df.head(5)

,Journal Name,ISSN,e-ISSN,DimensionsPubs,DimensionsIssns
0,3 Biotech,2190-572X,2190-5738,None,None
1,AAAS MEETING AND SCIENCE INNOVATION EXPOSITION,NaN,NaN,None,None
2,AAPS Journal,1550-7416,NaN,None,None
3,Abdominal Radiology,2366-004X,2366-0058,None,None
4,Abhandlung Naturhistorische Gesellschaft Nuern...,0077-6149,NaN,None,None


In [49]:
q = """search publications where journal.title="{}" return publications[id+issn] limit 1"""

In [ ]:
for i, r in df.iterrows():
    if i < 100000000: # lower this number eg for testing 
        title = r['Journal Name']
        print(title)
        res = dslquery(q.format(title))
        try:
            tot = res.stats['total_count']
            r['DimensionsPubs'] = tot
        except:
            print(res['errors'])
        try:
            issns = res.publications[0]['issn']
            r['DimensionsIssns'] = issns
        except:
            pass
        time.sleep(1) # to avoid overloading API 

3 Biotech
Returned Publications: 1 (total = 1819)
AAAS MEETING AND SCIENCE INNOVATION EXPOSITION
Returned Publications: 0
AAPS Journal
Returned Publications: 1 (total = 1822)
Abdominal Radiology
Returned Publications: 1 (total = 5302)
Abhandlung Naturhistorische Gesellschaft Nuernberg
Returned Publications: 0
Abhandlungen Der Senckenberg Gesellschaft Fur Naturforschung
Returned Publications: 0
Abhandlungen Naturwissenschaftlichen Verein Zu Bremen
Returned Publications: 0
Abstracts of Papers of the American Chemical Society
Returned Publications: 0
Abstracts of the General Meeting of the American Society for Microbiology
Returned Publications: 0
Abstracts of the Interscience Conference on Antimicrobial Agents and Chemotherapy
Returned Publications: 0
Acarina
Returned Publications: 1 (total = 65)
Acarologia
Returned Publications: 1 (total = 568)
Accident Analysis and Prevention
Returned Publications: 0
Acrocephalus
Returned Publications: 1 (total = 98)
ACS BIOMATERIALS SCIENCE & ENGINEER

In [ ]:
df.head()

,Journal Name,ISSN,e-ISSN,DimensionsPubs,DimensionsIssns
0,3 Biotech,2190-572X,2190-5738,1819,"[2190-572X, 2190-5738]"
1,AAAS MEETING AND SCIENCE INNOVATION EXPOSITION,NaN,NaN,0,None
2,AAPS Journal,1550-7416,NaN,1822,"[1234-1234, 1550-7416]"
3,Abdominal Radiology,2366-004X,2366-0058,5302,"[2366-004X, 2366-0058]"
4,Abhandlung Naturhistorische Gesellschaft Nuern...,0077-6149,NaN,0,None


---
Finally

In [ ]:
df.to_csv("output.csv")